In [16]:
sealed class DataResult<out T> {

    data class Success<out T>(val data: T) : DataResult<T>()

    sealed class Error(
        open val message: String,
        open val throwable: Throwable? = null,
    ) : DataResult<Nothing>() {

        sealed class Client(
            override val message: String,
            override val throwable: Throwable? = null,
        ) : Error(message, throwable) {

            data class Database(
                override val message: String = "Database error",
                override val throwable: Throwable? = null,
            ) : Client(message, throwable)

            data class Mapping(
                override val message: String = "Mapping error",
                override val throwable: Throwable? = null,
            ) : Client(message, throwable)

            data class UnexpectedError(
                override val message: String = "Unexpected client error",
                override val throwable: Throwable? = null,
            ) : Client(message, throwable)
        }

        sealed class Network(
            override val message: String,
            override val throwable: Throwable? = null,
        ) : Error(message, throwable) {

            data class HttpError(
                val code: Int,
                override val message: String = "HTTP error: $code",
                override val throwable: Throwable? = null,
            ) : Network(message, throwable)

            data class ConnectionError(
                override val message: String = "Network connection error",
                override val throwable: Throwable? = null,
            ) : Network(message, throwable)

            data class SerializationError(
                override val message: String = "Serialization error",
                override val throwable: Throwable? = null,
            ) : Network(message, throwable)

            data class UnknownError(
                override val message: String = "Unknown network error",
                override val throwable: Throwable? = null,
            ) : Network(message, throwable)
        }
    }
}

fun <T, R> DataResult<List<T>>.map(transform: (T) -> R): DataResult<List<R>> {
    return when (this) {
        is DataResult.Success -> DataResult.Success(this.data.map(transform))
        is DataResult.Error -> this
    }
}


In [17]:
import kotlinx.coroutines.flow.Flow

data class SendChatMessage(
    val message: String
)

data class Message (
    val message: String
)

interface MessengerRepository {
    fun createMessage(sendChatMessage: SendChatMessage): DataResult<String>
    fun getMessageListFlow(chatId: String): Flow<List<Message>>
}

In [18]:
import kotlinx.coroutines.flow.Flow


class CreateMessageUseCase(
    private val messengerRepository: MessengerRepository
) {

    suspend operator fun invoke(
        messageText: String
    ): UseCaseResult {

        val sendChatMessage = SendChatMessage(
            message = messageText
        )

        val result = messengerRepository.createMessage(sendChatMessage = sendChatMessage)

        when (result) {
            is DataResult.Success -> {
                return UseCaseResult.Success(sendChatMessage = sendChatMessage)
            }

            is DataResult.Error.Client -> {
                return UseCaseResult.Error.Client
            }

            is DataResult.Error.Network -> {
                return UseCaseResult.Error.Network
            }

        }
    }

    sealed interface UseCaseResult {
        data class Success(val sendChatMessage: SendChatMessage) : UseCaseResult
        sealed class Error(val message: String) : UseCaseResult {
            data object Client : Error("Unknown client error")
            data object Network : Error("Unknown network error")
        }
    }

}



In [20]:
import io.ktor.http.ContentType
import kotlinx.coroutines.flow.Flow

class GetChatMessageListFlowUseCase(
    private val messengerRepository: MessengerRepository,
) {

    operator fun invoke(chatId: String): Flow<List<Message>> {
        return messengerRepository.getMessageListFlow(chatId = chatId)
    }

}